In [1]:
import sys
sys.path.append("../src/")

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from os.path import join, exists
from sys import stdout
from keras.preprocessing.image import ImageDataGenerator

from global_config import FOLDER_IMAGES, FOLDER_MRKS, FILE_DATASET, IMAGE_SIZE, RANDOM_SEED
from icao_dataset import ICAODataset

%matplotlib inline

Using TensorFlow backend.


In [2]:
STAMP_SIZE = (50, 50)
BATCH_SIZE = 32

PATH_TO_ROOT = "../"
FILE_DATASET = join(PATH_TO_ROOT, FILE_DATASET)
FOLDER_DATA = join(PATH_TO_ROOT, "data")
FOLDER_MRKS = join(PATH_TO_ROOT, FOLDER_MRKS)
FOLDER_IMAGES = join(PATH_TO_ROOT, FOLDER_IMAGES)
FOLDER_INKMARKED = join(FOLDER_DATA, "inkmarked_creased")
FOLDER_INKMARKED_IMAGES = join(FOLDER_INKMARKED, "images")
FOLDER_INKMARKED_MRKS = join(FOLDER_INKMARKED, "ground_truth")

print(FILE_DATASET)
print(FOLDER_DATA)
print(FOLDER_IMAGES)
print(FOLDER_MRKS)
print(FOLDER_INKMARKED_IMAGES)
print(FOLDER_INKMARKED_MRKS)

../data/inkmarked_creased/dataset.pkl
../data
../data/cropped_faces/images/
../data/cropped_faces/ground_truth/
../data\inkmarked_creased\images
../data\inkmarked_creased\ground_truth


# Data Loading 

In [3]:
if exists(FILE_DATASET):
    dataset = pkl.load(open(FILE_DATASET, "rb"))
else:
    dataset = ICAODataset()
    dataset.load(FOLDER_IMAGES, FOLDER_MRKS, output_size=IMAGE_SIZE)
    pkl.dump(dataset, open(FILE_DATASET, "wb"))

5731 of 5731
5731 of 5731


In [4]:
N_IMAGES = len(dataset.images)
N_MRKS = len(dataset.mrks)

print(N_IMAGES)
print(N_MRKS)
assert N_IMAGES == N_MRKS 

5731
5731


# Image Data Generation

In [5]:
datagen = ImageDataGenerator(
    rotation_range=90,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

In [6]:
flow = datagen.flow_from_directory(
    FOLDER_DATA,
    target_size=STAMP_SIZE,
    color_mode="rgba",
    classes=["stamps"],
    batch_size=BATCH_SIZE,
    seed=RANDOM_SEED,
    interpolation="bicubic",
)

Found 32 images belonging to 1 classes.


In [7]:
def rect_contains(rect, point):
    top, left, bottom, right = rect
    x, y = point
    return top < y < (top + bottom) and left < x < (left + right)


def random_point_out_of_rect(rect, max_x, max_y):
    random_x, random_y = rect[0] + 1, rect[1] + 1
    while rect_contains(rect, (random_x, random_y)):
        random_x = np.random.choice(max_x)
        random_y = np.random.choice(max_y)
    return random_x, random_y


def create_foreground_image(stamp, height, width, inner_rect):
    im_fgd = np.zeros((height, width, 4))
    stamp_height, stamp_width = stamp.shape[0], stamp.shape[1]
    
    random_x, random_y = random_point_out_of_rect(
        inner_rect,
        width - stamp_width - 1, 
        height - stamp_height - 1,
    )
    
    top, left = random_y, random_x
    bottom, right = top + stamp_height, left + stamp_width
    im_fgd[top:bottom, left:right, :] = stamp
    return im_fgd


def alpha_blending(im_foreground, im_background):
    c1, c2, c3, ca = cv2.split(im_foreground)
    im_fgd = cv2.merge((c1, c2, c3)).astype(np.float)
    im_bgd = im_background.astype(np.float)
    alpha = cv2.merge((ca, ca, ca)).astype(np.float) / 255

    im_fgd = cv2.multiply(alpha, im_fgd)
    im_bgd = cv2.multiply(1.0 - alpha, im_bgd)
    im_out = cv2.add(im_fgd, im_bgd)
    return im_out


def add_stamp_to_image(image, stamp, inner_rect, border_size=25):
    im_bgd = cv2.copyMakeBorder(
        image,
        top=border_size,
        bottom=border_size,
        left=border_size,
        right=border_size,
        borderType=cv2.BORDER_CONSTANT,
        value=0,
    )
    
    im_fgd = create_foreground_image(stamp, im_bgd.shape[0], im_bgd.shape[1], inner_rect)
    im_out = alpha_blending(im_fgd, im_bgd)
    
    top = left = border_size
    bottom = top + image.shape[0]
    right = left + image.shape[1]
    
    return im_out[top:bottom, left:right]

In [8]:
INNER_RECT = (40, 40, 120, 120) # stamps will not be overlayed inside this rect
batch_idxs = np.split(range(N_IMAGES), range(BATCH_SIZE, N_IMAGES, BATCH_SIZE))
N_BATCHES = len(batch_idxs)

for i, idxs in enumerate(batch_idxs, start=1):
    print(f"{i} of {N_BATCHES}", end='\r')
    stdout.flush()
    
    half_size = len(idxs) // 2
    random_idxs_images = np.random.choice(idxs, size=half_size, replace=False)
    random_idxs_stamps = np.random.choice(BATCH_SIZE, size=half_size, replace=False)
    gen_stamps = flow.next()[0][random_idxs_stamps]
    
    for idx_img, stamp in zip(random_idxs_images, gen_stamps):
        dataset.images[idx_img] = add_stamp_to_image(dataset.images[idx_img], stamp, INNER_RECT)
        dataset.mrks[idx_img].photo_reqs.ink_marked_creased.value = 0

In [9]:
dataset.save(
    folder_images=FOLDER_INKMARKED_IMAGES,
    folder_mrks=FOLDER_INKMARKED_MRKS
)